# 03: Enriquecimiento Biográfico

**Propósito:** Este *notebook* enriquece la lista de diputados (`diputados.csv`) con datos biográficos detallados, creando `diputados_bio.csv`.

**Proceso:**
1.  Carga la lista de períodos (`periodos_master.csv`).
2.  Carga la tabla de consulta de BCN (`bcn_url_master_lookup.csv`).
3.  Para cada período:
    a. Carga `diputados.csv`.
    b. Resuelve las URLs de BCN usando *fuzzy matching* contra la tabla de consulta.
    c. Scrapea el texto de las biografías y el distrito desde las URLs encontradas.
    d. Usa un LLM (Ollama) para extraer datos estructurados (JSON) del texto.
    e. Guarda el `DataFrame` enriquecido como `diputados_bio.csv`.

**Dependencias:**
* `data/01_raw/periodos_master.csv` (de `00_Extraction_Periods`)
* `data/01_raw/bcn_url_master_lookup.csv` (de `02_Extraction_BCN_Lookup`)
* `data/01_raw/[periodo]/diputados.csv` (de `01_Extraction_Deputies`)

**Artefactos de Salida:**
* `data/01_raw/[periodo]/diputados_bio.csv`

In [1]:
import pandas as pd
from pathlib import Path
import sys
import logging
import json
from tqdm.notebook import tqdm # Para barras de progreso

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.extraction_utils import sanitize_filename
    from src.bio_utils import (
        find_best_match_bcn,    
        scrape_bcn_bio_data,       
        get_ollama_client, 
        extract_bio_data_llm
    )
except ImportError as e:
    logging.error(f"ERROR: No se pudieron importar las funciones desde /src. {e}")
    logging.error("Asegúrese de que 'src/extraction_utils.py' y 'src/bio_utils.py' existan y contengan las funciones.")
    raise

# Registrar 'tqdm' con pandas para usar .progress_apply()
tqdm.pandas()

In [2]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"

# --- Definir ARCHIVOS DE ENTRADA (Dependencias) ---
MASTER_PERIOD_FILE = DATA_DIR_RAW / "periodos_master.csv"
BCN_LOOKUP_FILE = DATA_DIR_RAW / "bcn_url_master_lookup.csv"

# --- Configuración del LLM ---
OLLAMA_HOST = 'http://127.0.0.1:11434'
OLLAMA_MODEL = 'llama3:instruct' # Modelo recomendado
FUZZY_MATCH_THRESHOLD = 70       # Umbral de confianza para el matching

logging.info(f"Ruta Raíz: {ROOT}")
logging.info(f"Directorio de Datos Raw: {DATA_DIR_RAW}")
logging.info(f"Modelo LLM: {OLLAMA_MODEL}")

2025-10-25 01:16:14,915 - INFO - Ruta Raíz: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-
2025-10-25 01:16:14,919 - INFO - Directorio de Datos Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw
2025-10-25 01:16:14,921 - INFO - Modelo LLM: llama3:instruct


## 2. Carga de Dependencias (Períodos, BCN Lookup, Cliente LLM)

Cargamos los artefactos maestros necesarios para el bucle y conectamos con Ollama.

In [3]:
try:
    df_periodos = pd.read_csv(MASTER_PERIOD_FILE)
    logging.info(f"Se cargó la lista maestra de {len(df_periodos)} períodos.")
    
    df_bcn_lookup = pd.read_csv(BCN_LOOKUP_FILE)
    logging.info(f"Se cargó la tabla de consulta BCN con {len(df_bcn_lookup)} registros.")
    
    # Crear la lista de períodos válidos para la función de scraping
    # (Corregido el bug de la variable global 'periodos')
    lista_periodos_validos = df_periodos['Nombre'].tolist()
    logging.info(f"Lista de {len(lista_periodos_validos)} períodos válidos creada.")

except FileNotFoundError as e:
    logging.error(f"ERROR FATAL: No se encontró el archivo de dependencia: {e.filename}")
    logging.error("Por favor, ejecute los notebooks '00' y '02' primero.")
    raise

# Inicializar cliente Ollama (UNA SOLA VEZ)
ollama_client = get_ollama_client(OLLAMA_HOST)
if ollama_client is None:
    logging.warning("No se pudo conectar a Ollama. La extracción de datos estructurados (LLM) se omitirá.")

2025-10-25 01:16:14,941 - INFO - Se cargó la lista maestra de 10 períodos.
2025-10-25 01:16:14,950 - INFO - Se cargó la tabla de consulta BCN con 623 registros.
2025-10-25 01:16:14,952 - INFO - Lista de 10 períodos válidos creada.
2025-10-25 01:16:15,369 - INFO - HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
2025-10-25 01:16:15,370 - INFO - Cliente Ollama conectado exitosamente en http://127.0.0.1:11434


In [4]:
logging.info(f"Iniciando enriquecimiento biográfico para {len(df_periodos)} períodos...")

choices_bcn = df_bcn_lookup['nombre_en_lista'].tolist()
url_map_bcn = df_bcn_lookup.set_index('nombre_en_lista')['url_wiki'].to_dict()

for row in tqdm(df_periodos[9:].itertuples(), total=len(df_periodos), desc="Procesando Períodos"):
    nombre_periodo = row.Nombre
    nombre_carpeta = sanitize_filename(nombre_periodo)
    carpeta_periodo = DATA_DIR_RAW / nombre_carpeta
    ruta_input_diputados = carpeta_periodo / "diputados.csv"
    ruta_output_bio = carpeta_periodo / "diputados_bio.csv"

    logging.info(f"--- Procesando Período: {nombre_periodo} ---")

    # --- Chequeos ---
    if not ruta_input_diputados.exists():
        logging.warning(f"Saltando período: No se encontró archivo de diputados en {ruta_input_diputados}")
        continue
    if ruta_output_bio.exists():
        logging.info(f"Saltando período: El archivo 'diputados_bio.csv' ya existe.")
        continue

    try:
        # --- 1. Cargar Datos y Crear 'nombre_completo' ---
        df_diputados = pd.read_csv(ruta_input_diputados)
        logging.info(f"Cargados {len(df_diputados)} diputados.")

        try:
            cols_nombre = ['Diputado.Nombre', 'Diputado.ApellidoPaterno', 'Diputado.ApellidoMaterno']
            df_diputados['nombre_completo'] = (
                df_diputados[cols_nombre[0]].astype(str) + ' ' + 
                df_diputados[cols_nombre[1]].astype(str) + ' ' + 
                df_diputados[cols_nombre[2]].astype(str)
            )
            df_diputados['nombre_completo'] = df_diputados['nombre_completo'].str.replace('nan', '', case=False).str.strip().str.replace(r'\s+', ' ', regex=True)
            logging.info("Columna 'nombre_completo' creada.")
        except KeyError as e:
            logging.error(f"ERROR: Faltan columnas de nombre clave {e}. No se puede hacer fuzzy matching.")
            continue 

        # --- 2. OPTIMIZACIÓN DE MATCHING ---
        logging.info("Optimizando: Creando lista de nombres únicos...")
        
        # Obtenemos los nombres únicos del DataFrame de diputados
        nombres_unicos = df_diputados['nombre_completo'].unique()
        logging.info(f"Se encontraron {len(nombres_unicos)} nombres únicos para {len(df_diputados)} filas.")
        
        df_queries = pd.DataFrame(nombres_unicos, columns=['nombre_completo'])

        tqdm.pandas(desc="Fuzzy Matching (Únicos)")
        match_results = df_queries['nombre_completo'].progress_apply(
            lambda nombre_query: find_best_match_bcn(
                nombre_query, 
                choices_bcn, 
                threshold=FUZZY_MATCH_THRESHOLD
            )
        )
        
        df_match = pd.DataFrame(
            match_results.tolist(), 
            index=df_queries.index, 
            columns=['match_nombre_bcn', 'match_score']
        )
        
        df_lookup_final = df_queries.join(df_match)
        df_lookup_final['url_wiki'] = df_lookup_final['match_nombre_bcn'].map(url_map_bcn)

        # --- 3. MAPEO DE VUELTA ---
        df_dip_matched = (pd.merge(
            df_diputados,
            df_lookup_final,
            on='nombre_completo',
            how='left' # Mantiene todos los diputados originales
        ).drop_duplicates(subset=['nombre_completo']))
        display(df_dip_matched)
        # --- 4. Scraping ---
        logging.info("Iniciando scraping de biografías BCN...")
        scraped_data_list = df_dip_matched['url_wiki'].progress_apply(
            lambda url: scrape_bcn_bio_data(url, lista_periodos_validos)
        )
        df_scraped = pd.DataFrame(scraped_data_list.tolist(), index=df_dip_matched.index)
        df_dip_scraped = df_dip_matched.join(df_scraped)
        display(df_dip_scraped.head(3))
        # --- 5. Extracción LLM ---
        logging.info("Iniciando extracción con LLM (Ollama)...")
        if ollama_client:
            bio_data_list = df_dip_scraped['bio_texto_completo'].progress_apply(
                lambda x: extract_bio_data_llm(ollama_client, x, OLLAMA_MODEL)
            )
            df_bio_extraida = pd.DataFrame(
                [data if isinstance(data, dict) else {} for data in bio_data_list],
                index=df_dip_scraped.index
            )
            df_final_enriquecido = pd.concat([df_dip_scraped, df_bio_extraida], axis=1)
        else:
            logging.warning("Omitiendo extracción LLM (cliente no disponible).")
            df_final_enriquecido = df_dip_scraped

        # --- 6. Guardar ---
        df_final_enriquecido.to_csv(ruta_output_bio, index=False, encoding="utf-8")
        logging.info(f"Guardado exitosamente: {ruta_output_bio}")

    except Exception as e:
        logging.error(f"ERROR FATAL al procesar período {nombre_periodo}: {e}", exc_info=True)

logging.info("--- Enriquecimiento biográfico finalizado ---")

2025-10-25 01:16:15,388 - INFO - Iniciando enriquecimiento biográfico para 10 períodos...


Procesando Períodos:   0%|          | 0/10 [00:00<?, ?it/s]

2025-10-25 01:16:15,423 - INFO - --- Procesando Período: 2022-2026 ---
2025-10-25 01:16:15,432 - INFO - Cargados 332 diputados.
2025-10-25 01:16:15,438 - INFO - Columna 'nombre_completo' creada.
2025-10-25 01:16:15,438 - INFO - Optimizando: Creando lista de nombres únicos...
2025-10-25 01:16:15,439 - INFO - Se encontraron 157 nombres únicos para 332 filas.


Fuzzy Matching (Únicos):   0%|          | 0/157 [00:00<?, ?it/s]

,FechaInicio,FechaTermino,Distrito,Diputado.Id,Diputado.Nombre,Diputado.Nombre2,Diputado.ApellidoPaterno,Diputado.ApellidoMaterno,Diputado.FechaNacimiento,Diputado.FechaDefucion,...,Diputado.Militancias.Militancia,FechaInicio.1,FechaTermino.1,Partido.Id,Partido.Nombre,Partido.Alias,nombre_completo,match_nombre_bcn,match_score,url_wiki
0,2026-03-10 23:59:59,NaN,NaN,74,Jaime,NaN,Naranjo,Ortiz,1951-01-12,NaN,...,"{'FechaInicio': datetime.datetime(2025, 7, 3, ...",2025-07-03 00:00:00,2026-03-10 23:59:59,IND,Independientes,IND,Jaime Naranjo Ortiz,Naranjo Ortiz Jaime,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
6,2026-03-10 23:59:59,NaN,NaN,803,René,NaN,Alinco,Bustos,1958-06-02,NaN,...,"{'FechaInicio': datetime.datetime(2006, 3, 11,...",2006-03-11 00:00:00,2010-03-10 23:59:59,PPD,Partido Por la Democracia,PPD,René Alinco Bustos,Alinco Bustos René,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
10,2026-03-10 23:59:59,NaN,NaN,815,Sergio,NaN,Bobadilla,Muñoz,1958-03-25,NaN,...,"{'FechaInicio': datetime.datetime(2010, 3, 11,...",2010-03-11 00:00:00,2014-03-10 23:59:59,UDI,Unión Demócrata Independiente,UDI,Sergio Bobadilla Muñoz,Bobadilla Muñoz Sergio Enrique,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
14,2026-03-10 23:59:59,NaN,NaN,872,Jaime,NaN,Mulet,Martínez,1963-08-03,NaN,...,"{'FechaInicio': datetime.datetime(1998, 3, 11,...",1998-03-11 00:00:00,2002-03-10 23:59:59,DC,Partido Demócrata Cristiano,DC,Jaime Mulet Martínez,Mulet Martínez Jaime,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
19,2026-03-10 23:59:59,NaN,NaN,917,Gastón,NaN,Von Mühlenbrock,Zamora,1954-12-26,NaN,...,"{'FechaInicio': datetime.datetime(2006, 3, 11,...",2006-03-11 00:00:00,2010-03-10 23:59:59,UDI,Unión Demócrata Independiente,UDI,Gastón Von Mühlenbrock Zamora,Von Mühlenbrock Zamora Gastón,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,2026-03-10 23:59:59,NaN,NaN,1181,Nelson,NaN,Venegas,Salazar,1974-04-27,NaN,...,"{'FechaInicio': datetime.datetime(2022, 3, 11,...",2022-03-11 00:00:00,2026-03-10 23:59:59,PS,Partido Socialista,PS,Nelson Venegas Salazar,Venegas Salazar Nelson Esteban,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
328,2026-03-10 23:59:59,NaN,NaN,1182,Sebastián,NaN,Videla,Castillo,1986-04-29,NaN,...,"{'FechaInicio': datetime.datetime(2022, 3, 11,...",2022-03-11 00:00:00,2026-03-10 23:59:59,IND,Independientes,IND,Sebastián Videla Castillo,Videla Castillo Sebastián Patricio,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
329,2026-03-10 23:59:59,NaN,NaN,1183,Flor,NaN,Weisse,Novoa,1960-03-27,NaN,...,"{'FechaInicio': datetime.datetime(2022, 3, 11,...",2022-03-11 00:00:00,2026-03-10 23:59:59,UDI,Unión Demócrata Independiente,UDI,Flor Weisse Novoa,Weisse Novoa Flor Isabel,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...
330,2026-03-10 23:59:59,NaN,NaN,1184,Roberto,NaN,Celedón,Fernández,1947-01-31,NaN,...,"{'FechaInicio': datetime.datetime(2025, 1, 8, ...",2025-01-08 00:00:00,2026-03-10 23:59:59,IND,Independientes,IND,Roberto Celedón Fernández,Celedón Fernández Roberto Antonio,100.0,https://www.bcn.cl/historiapolitica/resenas_pa...


...

2025-10-25 01:17:24,112 - INFO - Iniciando extracción con LLM (Ollama)...


Fuzzy Matching (Únicos):   0%|          | 0/157 [00:00<?, ?it/s]

2025-10-25 01:17:24,122 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,124 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,125 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,125 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,127 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,128 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,130 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,131 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,132 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inv

2025-10-25 01:17:24,208 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,210 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,210 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,211 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,212 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,213 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,213 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,216 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,217 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inv

2025-10-25 01:17:24,278 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,279 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,279 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,280 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,282 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,284 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,284 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,284 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inválido).
2025-10-25 01:17:24,286 - INFO - extract_bio_data_llm: Texto de biografía omitido (demasiado corto o inv

,status,distrito,familia_juventud_parrafos,estudios_vida_laboral_parrafos,bio_texto_completo
0,200,None,[],[],
6,200,None,[],[],
10,200,None,[],[],
14,200,None,[],[],
19,200,None,[],[],
...,...,...,...,...,...
327,200,None,[],[],
328,200,None,[],[],
329,200,None,[],[],
330,200,None,[],[],
